<a href="https://colab.research.google.com/github/gonwi/research/blob/master/bert_transformers_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./sample_data"))

# Any results you write to the current directory are saved as output.

['README.md', 'anscombe.json', 'mnist_test.csv', 'california_housing_train.csv', 'california_housing_test.csv', 'mnist_train_small.csv']


In [0]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import zipfile
from matplotlib import pyplot as plt
%matplotlib inline
import sys
import datetime

In [3]:
!wget https://www.dropbox.com/s/8jnulb2l4v7ikir/model.zip

--2019-10-30 05:18:39--  https://www.dropbox.com/s/8jnulb2l4v7ikir/model.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/8jnulb2l4v7ikir/model.zip [following]
--2019-10-30 05:18:45--  https://www.dropbox.com/s/raw/8jnulb2l4v7ikir/model.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6f27098ea06ecbc17dcd2d4709.dl.dropboxusercontent.com/cd/0/inline/AraAzB2q-7Re4a0Hq89PnMNeuuO1fuGeEkdftPxAjD3_fwBwHDg0PTqD7eTsyRLW98IxGM603AZMMLUEwzad6ihALHchmTuzR_1tkNpPnczCFz5eckpEen-7CREuUBuXf64/file# [following]
--2019-10-30 05:18:45--  https://uc6f27098ea06ecbc17dcd2d4709.dl.dropboxusercontent.com/cd/0/inline/AraAzB2q-7Re4a0Hq89PnMNeuuO1fuGeEkdftPxAjD3_fwBwHDg0PTqD7eTsyRLW98IxGM603AZMMLUEwzad6ihALHchmTuzR_1tkNpPnczCFz5eckpEen-7CRE

In [0]:
repo = 'model_repo'
with zipfile.ZipFile("model.zip","r") as zip_ref:
    zip_ref.extractall(repo)

In [5]:
!ls 'model_repo/model'

bert_config.json  pytorch_model.bin  vocab.txt


In [0]:
!mv 'model_repo/model/bert_config.json' 'model_repo/model/config.json'


In [7]:
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/run_squad.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py

--2019-10-30 05:20:42--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [text/plain]
Saving to: ‘modeling.py’

modeling.py         100%[===================>]  37.03K  --.-KB/s    in 0.01s   

2019-10-30 05:20:42 (2.99 MB/s) - ‘modeling.py’ saved [37922/37922]

--2019-10-30 05:20:44--  https://raw.githubusercontent.com/google-research/bert/master/optimization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6258 (6.1K) [text/plain]
Saving to: ‘optimizati

In [8]:
BERT_MODEL = 'model'
BERT_PRETRAINED_DIR = f'{repo}/model'
OUTPUT_DIR = f'{repo}/outputs'
print(f'***** Model output directory: {OUTPUT_DIR} *****')
print(f'***** BERT pretrained directory: {BERT_PRETRAINED_DIR} *****')

***** Model output directory: model_repo/outputs *****
***** BERT pretrained directory: model_repo/model *****


In [9]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json


--2019-10-30 05:21:23--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘train-v1.1.json’

train-v1.1.json     100%[===================>]  28.88M   108MB/s    in 0.3s    

2019-10-30 05:21:24 (108 MB/s) - ‘train-v1.1.json’ saved [30288272/30288272]

--2019-10-30 05:21:25--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6M) [application/json]
Saving to: ‘dev-v1.1.json’

dev-v1.1.json       1

In [10]:
!wget https://github.com/allenai/bi-att-flow/blob/master/squad/evaluate-v1.1.py

--2019-10-30 05:21:41--  https://github.com/allenai/bi-att-flow/blob/master/squad/evaluate-v1.1.py
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘evaluate-v1.1.py’

evaluate-v1.1.py        [ <=>                ]  92.43K  --.-KB/s    in 0.04s   

2019-10-30 05:21:41 (2.40 MB/s) - ‘evaluate-v1.1.py’ saved [94650]



In [0]:
!mkdir output

In [12]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/utils_squad.py


--2019-10-30 05:22:13--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/utils_squad.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41521 (41K) [text/plain]
Saving to: ‘utils_squad.py’

utils_squad.py      100%[===================>]  40.55K  --.-KB/s    in 0.01s   

2019-10-30 05:22:13 (3.26 MB/s) - ‘utils_squad.py’ saved [41521/41521]



In [13]:
!pip install transformers

     |████████████████████████████████| 317kB 2.8MB/s 
     |████████████████████████████████| 655kB 46.1MB/s 
     |████████████████████████████████| 1.0MB 42.3MB/s 
     |████████████████████████████████| 860kB 40.8MB/s 
  Created wheel for regex: filename=regex-2019.8.19-cp36-cp36m-linux_x86_64.whl size=609228 sha256=6f1000f9046e42deb166546c4ed29586ab6316d2b52166f9d8298afee2ec713c
  Stored in directory: /root/.cache/pip/wheels/90/04/07/b5010fb816721eb3d6dd64ed5cc8111ca23f97fdab8619b5be
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=40c8c7ae98821097427357182eb9ca4d65ef48813d8857c3a62b47ab22b9ee6c
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built regex sacremoses


In [14]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/utils_squad_evaluate.py

--2019-10-30 05:22:58--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/utils_squad_evaluate.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12493 (12K) [text/plain]
Saving to: ‘utils_squad_evaluate.py’

utils_squad_evaluat 100%[===================>]  12.20K  --.-KB/s    in 0s      

2019-10-30 05:22:58 (129 MB/s) - ‘utils_squad_evaluate.py’ saved [12493/12493]



In [18]:
!wget https://gist.githubusercontent.com/W4ngatang/60c2bdb54d156a41194446737ce03e2e/raw/17b8dd0d724281ed7c3b2aeeda662b92809aadd5/download_glue_data.pyb

--2019-10-30 05:50:20--  https://gist.githubusercontent.com/W4ngatang/60c2bdb54d156a41194446737ce03e2e/raw/17b8dd0d724281ed7c3b2aeeda662b92809aadd5/download_glue_data.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8225 (8.0K) [text/plain]
Saving to: ‘download_glue_data.py’

download_glue_data. 100%[===================>]   8.03K  --.-KB/s    in 0s      

2019-10-30 05:50:20 (111 MB/s) - ‘download_glue_data.py’ saved [8225/8225]



In [19]:
!python download_glue_data.py

	Completed!
	Completed!
Processing MRPC...
Local MRPC data not specified, downloading data from https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!


In [24]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/run_glue.py

--2019-10-30 05:54:06--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/run_glue.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28812 (28K) [text/plain]
Saving to: ‘run_glue.py’

run_glue.py         100%[===================>]  28.14K  --.-KB/s    in 0.01s   

2019-10-30 05:54:06 (2.27 MB/s) - ‘run_glue.py’ saved [28812/28812]



In [43]:


!python run_glue.py \
  --model_type bert \
  --model_name_or_path bert-base-cased-finetuned-mrpc \
  --task_name MRPC \
  --do_train \
  --do_eval \
  --do_lower_case \
  --data_dir glue_data/MRPC/ \
  --max_seq_length 128 \
  --per_gpu_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir /tmp/MRPC/ \
  --overwrite_output_dir \

10/30/2019 06:15:36 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
10/30/2019 06:15:36 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-finetuned-mrpc-config.json not found in cache or force_download set to True, downloading to /tmp/tmpqj3daom5
100% 340/340 [00:00<00:00, 181340.71B/s]
10/30/2019 06:15:36 - INFO - transformers.file_utils -   copying /tmp/tmpqj3daom5 to cache at /root/.cache/torch/transformers/70bd73627499d3e6af9210cb6b919b8288952641f5f19da57fa5b72a9938781e.a5e28732f8abfe06de5915d3f940c4e7b5c10ca3768dc480278011244dec7498
10/30/2019 06:15:36 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/70bd73627499d3e6af9210cb6b919b8288952641f5f19da57fa5b72a9938781e.a5e28732f8abfe06de5915d3f940c4e7b5c10ca3768dc480278011244dec7498
10/30/2019 06:15:36 - INFO - transformers.file_utils -   removing temp file /tmp/tm

In [0]:
!python3 run_squad.py \
--model_type bert \
--do_lower_case \
--model_name_or_path bert_model/ \
--train_file=train-v1.1.json \
--do_eval \
--predict_file=dev-v1.1.json \
--output_dir=bert_model/ \
--overwrite_output_dir \


10/30/2019 07:47:33 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
10/30/2019 07:47:33 - INFO - transformers.configuration_utils -   loading configuration file bert_model/config.json
10/30/2019 07:47:33 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "pruned_heads": {},
  "torchscript": false,
  "

In [39]:
from IPython.display import FileLink
FileLink(r'/tmp/')

ValueError: ignored

In [0]:
from google.colab import files
files.download('glue_data/MRPC/cached_dev_bert-base-cased_128_mrpc') 

In [0]:
from IPython.display import FileLink
FileLink(r'model_repo/model/nbest_predictions_.json')

In [55]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-whole-word-masking-finetuned-squad-pytorch_model.bin

--2019-10-30 07:39:28--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-whole-word-masking-finetuned-squad-pytorch_model.bin
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.237.237
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.237.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1334424802 (1.2G) [application/octet-stream]
Saving to: ‘bert-large-cased-whole-word-masking-finetuned-squad-pytorch_model.bin’

bert-large-cased-wh 100%[===================>]   1.24G  48.7MB/s    in 27s     

2019-10-30 07:39:55 (47.8 MB/s) - ‘bert-large-cased-whole-word-masking-finetuned-squad-pytorch_model.bin’ saved [1334424802/1334424802]



In [57]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-whole-word-masking-finetuned-squad-config.json

--2019-10-30 07:42:14--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-whole-word-masking-finetuned-squad-config.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.10.21
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.10.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 532 [application/json]
Saving to: ‘bert-large-cased-whole-word-masking-finetuned-squad-config.json’

bert-large-cased-wh 100%[===================>]     532  --.-KB/s    in 0s      

2019-10-30 07:42:14 (11.9 MB/s) - ‘bert-large-cased-whole-word-masking-finetuned-squad-config.json’ saved [532/532]



In [61]:
!ls

bert-large-cased-whole-word-masking-finetuned-squad-pytorch_model.bin
bert-large-cased-whole-word-masking-finetuned-squad-vocab.txt
bert_model
cached_dev_bert-large-cased-whole-word-masking-finetuned-squad_384
cached_train_model_384
dev-v1.1.json
download_glue_data.py
evaluate-v1.1.py
glue_data
modeling.py
model_repo
model.zip
optimization.py
output
__pycache__
run_glue.py
runs
run_squad.py
sample_data
tokenization.py
train-v1.1.json
utils_squad_evaluate.py
utils_squad.py


In [0]:
!mv bert_model/squad_vocab.txt bert_model/vocab.txt

In [65]:
!ls bert_model/

config.json  pytorch_model.bin	squad_vocab.txt
